In [4]:
import os
import glob
import numpy as np
import pandas as pd
from pycocotools.coco import COCO
from tqdm.auto import tqdm

root = "./datasets/reef/train"
root = os.path.abspath(root)
os.system(f"mkdir {root}/labels")

gts = glob.glob(os.path.join(root, "annotations/fold_*.json"))

In [9]:
for gt in gts:
    gt = COCO(gt)
    cats = {_: i for i, _ in enumerate(gt.cats)}
    for img_id in tqdm(gt.imgs):
        anns = []
        ann_ids = gt.getAnnIds(imgIds = [img_id])
        label_name = os.path.join(root, "labels", gt.imgs[img_id]["file_name"][:-4] + ".txt")
        os.system(f"mkdir -p {os.path.dirname(label_name)}")
        for ann_id in ann_ids:
            ann = gt.anns[ann_id]
            anns.append([cats[ann["category_id"]], *ann["bbox"]])
        if len(anns) > 0:
            anns = pd.DataFrame(anns)
            anns.iloc[:,[1,3]] /= gt.imgs[img_id]["width"]
            anns.iloc[:,[2,4]] /= gt.imgs[img_id]["height"]
            anns.iloc[:,[1,2]] += np.array(anns.iloc[:,[3,4]] / 2)
            anns.to_csv(label_name, index = False, header = False, sep = " ")
        else:
            os.system(f"touch {label_name}")


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


  0%|          | 0/4030 [00:00<?, ?it/s]

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


  0%|          | 0/3969 [00:00<?, ?it/s]

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


  0%|          | 0/7680 [00:00<?, ?it/s]

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


  0%|          | 0/2599 [00:00<?, ?it/s]

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


  0%|          | 0/5223 [00:00<?, ?it/s]

In [10]:
import yaml
os.system(f"mkdir -p {os.path.join(root, 'yolos')}")
for gt in gts:
    with open(os.path.join(root, "yolos", os.path.basename(gt).replace("json", "txt")), "w") as f:
        gt = COCO(gt)
        for img in gt.imgs.values():
            img_path = os.path.join(root, "images", img["file_name"])
            f.write(img_path + "\n")

for fold in range(5):
    data = {
        "train": [os.path.join(root, "yolos", f"fold_{_}.txt") for _ in range(5) if _ != fold],
        "val": os.path.join(root, "yolos", f"fold_{fold}.txt"),

        "nc": len(gt.cats),
        "names": [_['name'] for _ in gt.cats.values()]
    }
    with open(os.path.join(root, "yolos", f"fold_{fold}.yaml"), "w") as f:
        yaml.dump(data, f)
            
    

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
loading annotations into memory...
Done (t=0.13s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
